In [1]:
import os
import json
import torch

import numpy as np
from scipy.special import softmax
from numpy import argmax

from transformers import BertTokenizer, BertModel, BertConfig
from datetime import datetime
from torch.utils.data import Dataset, DataLoader


class FactoryEncoder:

    def __init__(self, tokenizer, device, max_length):
        self.tokenizer = tokenizer
        self.device = device
        self.max_length = max_length
    # end

    # Q: WHY NOT USING PARAMETERS?
    # A: FOR FUN
    def get_instance(self, sentences):
        instance = SimpleEncoder(sentences)
        instance.set_tokenizer(self.tokenizer)
        instance.set_device(self.device)
        instance.set_max_length(self.max_length)
        return instance
    # end
# end

class SimpleEncoder(Dataset):
    def __init__(self, sentences):
        self.sentences = sentences
    # end

    def __getitem__(self, index):
        sentence = self.sentences[index]
        sentence = ' '.join(sentence.split())
        inputs = self.tokenizer.encode_plus(
            sentence, None,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_tensors='pt'
        )

        for key in inputs:
            inputs[key] = inputs[key].to(self.device)
        # end

        return inputs
    # end

    def set_tokenizer(self, tokenizer):
        self.tokenizer = tokenizer
    # end

    def set_device(self, device):
        self.device = device
    # end

    def set_max_length(self, max_length):
        self.max_length = max_length
    # end


class SimpleBertClassifier(torch.nn.Module):

    DEFAULT_FILENAME_CLASSIFIER = '.model.json'
    DEFAULT_FILENAME_BERT = 'bert_config.json'
    DEFAULT_FILENAME_MODEL = 'model.pt'
    DEFAULT_KEYS_IGNORED_CLASSIFIER = ['metrics', 'allmetrics']

    def __init__(self, path_folder_model=None):
        super(SimpleBertClassifier, self).__init__()

        filename_config_classifier = self.__class__.DEFAULT_FILENAME_CLASSIFIER
        filename_config_bert = self.__class__.DEFAULT_FILENAME_BERT
        filename_model = self.__class__.DEFAULT_FILENAME_MODEL
        keys_ignored_classifier = self.__class__.DEFAULT_KEYS_IGNORED_CLASSIFIER


        self.path_folder_model = path_folder_model
        self.path_config_bert = os.path.join(path_folder_model, filename_config_bert)
        self.path_config_classifier = os.path.join(path_folder_model, filename_config_classifier)
        self.path_file_model = os.path.join(path_folder_model, filename_model)

        with open(self.path_config_classifier, 'r') as file:
            config_classifier = json.load(file)
        # end

        for key in keys_ignored_classifier:
            del(config_classifier[key])
        # end

        # classfier parameters
        self.classifier_input_size = config_classifier.get('bert').get('input_size')
        self.classifier_max_length = config_classifier.get('bert').get('max_length')
        self.classifier_output_size = config_classifier.get('bert').get('output_size')
        #

        self.labels_output_classifier = config_classifier.get('classes')
        self.dict_label_index = {label: index for index, label in enumerate(self.labels_output_classifier)}

        self.l1 = None
        self.linear = None

        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.loaded = False
    # end

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None):
        output_bert = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_bert[0]
        pooler = hidden_state[:, 0]
        output = self.classifier(pooler)
        return output
    # end

    def load(self):
        if not self.loaded:
            print('loading BERT from: {}'.format(self.path_config_bert))
            self.l1 = BertModel(BertConfig.from_pretrained(self.path_config_bert))
            print('loading {} x {} linear classfier layer'.format(self.classifier_input_size, self.classifier_output_size))
            self.classifier = torch.nn.Linear(self.classifier_input_size, self.classifier_output_size)

            self.to(self.device)
            self.load_state_dict(torch.load(self.path_file_model, map_location=torch.device(self.device)))
            self.l1.myname = 'jinyuj'

            print('loading Tokenizer from: {}'.format(self.path_folder_model))
            self.tokenizer = BertTokenizer.from_pretrained(self.path_folder_model)
            self.factory_encoder = FactoryEncoder(self.tokenizer, self.device, self.classifier_max_length)
            self.loaded = True
        # end

        return self
    # end

    def predicts(self, samples_input):
        encoder = self.factory_encoder.get_instance(samples_input)
        outputs_raw = []
        for sample_encoded in encoder:
            with torch.no_grad():
                output_raw = self(**sample_encoded).cpu().numpy().flatten().tolist()
                # print('jinyuj: predicts:83 output_raw: {}'.format(output_raw))
                outputs_raw.append(output_raw)
            # end
        # end

        info_result = {
            'outputs': outputs_raw,
            'labels': self.labels_output_classifier
        }

        return info_result
    # end
# end

In [2]:
import os, errno
import json
import yaml
import argparse
import sys
import traceback
import time
import pandas as pd
import torch
import torch.utils.data as data_utils
import transformers
from datetime import datetime
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, BertConfig
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# default constants

# The name of the bert model to load
VBERT_MODEL = "vBERT-2020-Base"

# vBERT-Base's hidden layer size is 768
# vBERT-Large's hidden layer size is 1024
CLASSIFICATION_LAYER_WIDTH = 768 if "Base" in VBERT_MODEL else 1024

# Maximum string length
# Anything longer will be truncated
# Maximum length can not exceed 512, though generally can be shorter
MAX_STRING_LEN = 128


# ## Class for transforming text for input into BERT
# 
# This class tokenizes the text and converts the tokens into tensors of token ids with an attention mask, along with the string's class label

class Prepare(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        title = str(self.data[1][index])
        title = " ".join(title.split())
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        label = self.data[0][index]

        target = torch.tensor(label, dtype=torch.long)

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': label
        }

    def __len__(self):
        return self.len

# ## vBERT Class for classification
# 
# To use vBERT for classification, you have to add a classification layer on top of the language model. 
# vBERT encodes the data.  The classification layer receives the encoding from vBERT and claculates class probabilities.

class BertClassifier(torch.nn.Module):
    # Create a classification network that uses vBERT to encode the input and pass to a classification layer
    #
    # Input configurable parameters:
    # root: full path name of the model directory, where the .pt, vocab and .model.json/bert_config.json file are found
    # name: name of the model (for reporting/diagnostic)
    # input_size: width of classification layer (e.g. 768)
    # output_size: number of output classes (task dependent)
    # max_length: maximum number of characters in input string for trainig/inference
    # classes: array of output_size with class names

    def __init__(self, config):
        super(BertClassifier, self).__init__()
        self.config = config
        self.model_dir = config.get("root", ".")
        self.model_name = config.get("name", VBERT_MODEL)
        self.classification_layer_width = config.get("input_size", CLASSIFICATION_LAYER_WIDTH)
        self.classes = config.get("classes", ["0", "1"])
        self.num_classes = config.get("output_size", len(self.classes))
        self.max_string_length = config.get("max_length", MAX_STRING_LEN)
        self.bert_config_file = self.model_dir + "/bert_config.json"
        self.model_file_path = self.model_dir + "/model.pt"
        self.device = config.get("device", "cpu")
        self.metrics = [ "accuracy", "precision", "recall", "F-score" ]

        # Load the vBERT vocabulary into the tokenizer
        print('Loading Tokenizer from: {}'.format(self.model_dir))
        self.tokenizer = BertTokenizer.from_pretrained(self.model_dir)

    def load_bert_for_training(self):
        print('Loading vBERT model: ' + self.model_name)
        self.l1 = BertModel.from_pretrained(pretrained_model_name_or_path=self.config.bert_model_dir)
        print("Adding {}x{} classification layer".format(self.classification_layer_width, self.num_classes))
        self.classifier = torch.nn.Linear(self.classification_layer_width, self.num_classes)

    def load_bert_for_inference(self):
        print('Loading vBERT config from: {}'.format(self.bert_config_file))
        self.l1 = BertModel(BertConfig.from_pretrained(self.bert_config_file))
        print("Adding {}x{} classification layer".format(self.classification_layer_width, self.num_classes))
        self.classifier = torch.nn.Linear(self.classification_layer_width, self.num_classes)

    # Encode the input with vBERT, read output from the last layer of vBERT, and pass to the classification layer
    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        output = self.classifier(pooler)
        return output

    def do_inference(self, data_loader, test_output=None):
        model = self.eval()
        predictions = []
        real_values = []
        with torch.no_grad():
            for data in data_loader:
                input_ids = data['ids'].to(self.device)
                attention_mask = data['mask'].to(self.device)
                targets = data['targets'].to(self.device)
                outputs = model(input_ids, attention_mask)
                _, preds = torch.max(outputs, dim=1)
                predictions.extend(preds)
                real_values.extend(targets)
                if test_output != None:
                    test_output.extend(outputs)
        predictions = torch.stack(predictions).cpu()
        real_values = torch.stack(real_values).cpu()

        test_accu = 100 * accuracy_score(real_values, predictions)
        test_precision, test_recall, test_fscore, ignore = precision_recall_fscore_support(real_values, predictions, average='macro')
        test_precision *= 100
        test_recall *= 100
        test_fscore *= 100
        metrics = [ test_accu, test_precision, test_recall, test_fscore ]
        return predictions, real_values, metrics

    def classify_text(self, txt: str, conf=None):
        dataset = pd.DataFrame.from_dict({ 'row': [ 0, txt ]}, orient='index')
        prepared_set = Prepare(dataset, self.tokenizer, self.max_string_length)
        params = {'batch_size': 1,
                    'shuffle': False,
                    'num_workers': 0
                }
        loader = DataLoader(prepared_set, **params)
        outputs = []
        predictions, real_values, metrics = self.do_inference(loader, outputs)
        classes = self.classes
        result = { 'classes': [ ], 'text': txt, 'top_class': classes[int(predictions[0])], 'top_class_index': int(predictions[0]) }
        xi = 0
        for x in classes:
            result['classes'].append({ 'class_name': x, 'confidence': float(outputs[0][xi]) })
            xi += 1
        return result

class InstaMLConfig():
    def __init__(self, config_json_file):
        with open(config_json_file, "r") as cjf:
            self.jconf = json.load(cjf)
        pth, fname = os.path.split(config_json_file)
        if len(pth) < 1:
            pth = "."
        self.name = self.jconf.get('name', '')
        self.model_dir = pth

    def load_instaML_model_file(self):
        model_info_file_name = self.model_dir + "/.model.json"
        with open(model_info_file_name, "r") as mif:
            self.model_info = json.load(mif)
        return self.model_info

    def save_instaML_model_file(self):
        model_info_file_name = self.model_dir + "/.model.json"
        with open(model_info_file_name, "w") as mif:
            json.dump(self.model_info, mif, indent=4)
        # end
    # end
# end


def get_config(path_config):

    configPath = path_config
    print(configPath)

    with open(configPath, "r") as yf:
        config = yaml.safe_load(yf)
    return config
# end


class InstaMLConfig():
    def __init__(self, config_json_file):
        with open(config_json_file, "r") as cjf:
            self.jconf = json.load(cjf)
        pth, fname = os.path.split(config_json_file)
        if len(pth) < 1:
            pth = "."
        self.name = self.jconf.get('name', '')
        self.model_dir = pth

    def load_instaML_model_file(self):
        model_info_file_name = self.model_dir + "/.model.json"
        with open(model_info_file_name, "r") as mif:
            self.model_info = json.load(mif)
        return self.model_info

    def save_instaML_model_file(self):
        model_info_file_name = self.model_dir + "/.model.json"
        with open(model_info_file_name, "w") as mif:
            json.dump(self.model_info, mif, indent=4)
        # end
    # end
# end

def load_model_readymlbert(conf, device):
    conf.load_instaML_model_file()

    bert_config = { 'root': conf.model_dir, 'name': conf.model_info['name'], 'classes': conf.model_info['classes'], 'device': device }
    bert_config.update(conf.model_info['bert'])
    model = BertClassifier(bert_config)
    model.load_bert_for_inference()
    model.to(device)
    model.load_state_dict(torch.load(model.model_file_path, map_location=torch.device(device)))
    model.l1.myname = 'vbert'
    return model
# end

def load_model(env_config, dir_model, device):
    modelDir = dir_model
    if not os.path.exists(modelDir):
        return None, "Model does not exist: " + modelDir

    print("Importing model from", modelDir)
    try:
        conf = InstaMLConfig(modelDir + "/.model.json")
        model = load_model_readymlbert(conf, device)
        return model, None
    except Exception as e:
        err = "Cannot load model: " + str(e)
        return None, err
    # end
# end

def predict(model, text):
    result = model.classify_text(text)
    return result
# end

def main_load_model(path_config='config.yaml', dir_model='vBERT-base-20', device='cuda'):
    env_config = get_config(path_config)
    model, err = load_model(env_config, dir_model, device)
    if not err:
        return model
    else:
        raise Exception(err)
    # end
# end



In [3]:
# jinyuj = SimpleBertClassifier('vBERT-base-20')
# jinyuj.load()
# model_jinyuj = jinyuj.l1
# model_jinyuj.myname = 'jinyuj'


vbert = main_load_model()
model_vbert = vbert.l1
model_vbert.myname = 'vbert'

config.yaml
Importing model from vBERT-base-20
Loading Tokenizer from: vBERT-base-20
Loading vBERT config from: vBERT-base-20/bert_config.json
Adding 768x16 classification layer


In [4]:
# tokenizer_jinyuj = jinyuj.factory_encoder.tokenizer
# tokenizer_jinyuj.myname = 'jinyuj'

tokenizer_vbert = vbert.tokenizer
tokenizer_vbert.myname = 'vbert'

In [5]:
def encode_bad(sentence, tokenizer, device='cuda'):
    sentence = " ".join(sentence.split())
    inputs = tokenizer.encode_plus(
        sentence,
        None,
        add_special_tokens=True,
        max_length=512,
        padding='max_length',
        return_token_type_ids=True,
        truncation=True,
        return_tensors='pt'
    )
    for key in inputs.keys():
        inputs[key] = inputs[key].to(device)
    # end
    
    return inputs
# end

In [6]:
def encode_bad2(sentence, tokenizer, model, device='cuda'):
    dataset = pd.DataFrame.from_dict({ 'row': [ 0, sentence ]}, orient='index')
    prepared_set = Prepare(dataset, tokenizer, 128)
    params = {'batch_size': 1,'shuffle': False,'num_workers': 0}
    loader = DataLoader(prepared_set, **params)
    with torch.no_grad():
        for data in loader:
            input_ids = data['ids'].to(device)
            attention_mask = data['mask'].to(device)
            outputs = model(input_ids, attention_mask)
        # for
    # with
    return outputs
# end

In [7]:
def encode_bad3(txt, tokenizer, model, device='cuda'):
    dataset = pd.DataFrame.from_dict({ 'row': [ 0, txt ]}, orient='index')
    prepared_set = Prepare(dataset, tokenizer, 128)
    params = {'batch_size': 1,
                'shuffle': False,
                'num_workers': 0
            }
    data_loader = DataLoader(prepared_set, **params)

    with torch.no_grad():
        for data in data_loader:
            input_ids = data['ids'].to(device)
            attention_mask = data['mask'].to(device)

            outputs_raw = model.l1(input_ids, attention_mask)  # model here is l1
        # end
    # end

    return outputs_raw


In [11]:
sentence = 'timestamp failed at play deploy vm efi nvme vmxnet number timestamp task upload local file to esxi data store task path home worker workspace dw rhel number arm ansible vsphere gos validation common esxi upload data store file yml number exception in vsphere copy python when main in request python when http error default fatal localhost failed http error number not found timestamp task testing exit due to failure task path home worker workspace dw rhel number arm ansible vsphere gos validation common test rescue yml number fatal localhost failed exit testing when exit testing when fail is set to true in test case deploy vm efi nvme vmxnet number'
outputs = encode_bad3(sentence, vbert.tokenizer, vbert)
print(outputs[0])

tensor([[[ 0.8497, -0.8521,  0.2835,  ...,  0.2763,  0.0888,  0.3067],
         [-0.1570,  1.1183, -0.7084,  ..., -0.6026, -0.2311,  0.3412],
         [ 0.0698,  0.1330, -0.3425,  ..., -0.5510,  0.0676, -0.7215],
         ...,
         [-0.0167,  0.1184,  0.0677,  ..., -0.2849, -0.1566,  0.6619],
         [-0.1870,  0.1368, -0.0331,  ..., -0.2738, -0.1185,  0.5610],
         [-0.0327,  0.1657, -0.2367,  ...,  0.0413,  0.0374,  0.6371]]],
       device='cuda:0')


In [9]:

# sentence = 'timestamp failed at play deploy vm efi nvme vmxnet number timestamp task upload local file to esxi data store task path home worker workspace dw rhel number arm ansible vsphere gos validation common esxi upload data store file yml number exception in vsphere copy python when main in request python when http error default fatal localhost failed http error number not found timestamp task testing exit due to failure task path home worker workspace dw rhel number arm ansible vsphere gos validation common test rescue yml number fatal localhost failed exit testing when exit testing when fail is set to true in test case deploy vm efi nvme vmxnet number'
# item_encoded = encode_bad(sentence, tokenizer_vbert)
# with torch.no_grad():
#     output = model_vbert(**item_encoded)
# # end
# output

